In [2]:
!pip install transformers

import pandas as pd
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer,T5Config
import warnings
warnings.filterwarnings(action='ignore')

     |████████████████████████████████| 890kB 4.8MB/s 
     |████████████████████████████████| 890kB 22.8MB/s 
     |████████████████████████████████| 3.0MB 31.1MB/s 
     |████████████████████████████████| 1.1MB 41.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5d78d2a7a3126e013dff44f42c12495d79bd66698940ea0758e676d780270c5b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:

import pandas as pd
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer,T5Config
import warnings
warnings.filterwarnings(action='ignore')

In [38]:
text='''
Pablo Casado se pronuncio finalmente sobre la denominada pieza Kitchen, referida al supuesto espionaje del PP a Luis Barcenas y sobre la que el juez ha levantado secreto de sumario, asegurando que son investigaciones judiciales que vienen de tiempos pasados y que reafirman su compromiso con la honorabilidad y la ejemplaridad que deben tener todos los partidos politicos”. El ahora lider de la formacion recordo que en 2012 y 2013 (años a los que se circunscriben las indagaciones) el era un diputado por avila que no pertenecia a la direccion.
Ademas, el lider del PP insistio en un foro organizado por el diario El Mundo que las personas sobre las que la Fiscalia pide una investigacion, entre los que estan el exministro del Interior, Jorge Fernandez Diaz, y la ex secretaria general del PP, Maria Dolores de Cospedal, estan fuera de la politica y no ostentan ninguna responsabilidad en el partido. Ni siquiera fueron en las listas electorales. Cospedal, de hecho, compitio con Casado en las primarias para liderar la formacion conservadora y decidio marcharse tras su fracaso.
Despues de varios dias son informaciones sobre los wasaps de Francisco Martinez, entonces secretario de Estado de Interior, en los que da por hecho que las ordenes que recibio y plasmadas en los mensajes podrian provocar no solo la imputacion del exministro, sino tambien la citacion del propio Mariano Rajoy, el actual lider del PP pidio reafirmar y defender la presuncion de inocencia, recordando que “muchas investigaciones cuentan con eco periodistico en sus fases iniciales” y luego quedan en nada. Los famosos ordenadores, por ejemplo. El PP quedo absuelto aunque ya nadie se acuerde, dijo Casado
'''

In [39]:
text1='''
El secretario general del PSOE de Madrid, Jose Manuel Franco, ha afirmado este martes que estudiarian proponer a una persona no socialista para la Presidencia de la Comunidad de Madrid en una futurible mocion de censura con tal de desalojar a la actual presidenta, Isabel Diaz Ayuso, de la Puerta del Sol. En principio nosotros hemos ganado las elecciones y se nos haria muy dificil este escenario, pero si la higiene democratica nos lleva a esa exigencia de tener que pasar porque una persona no socialista presida la Comunidad y eso nos conlleva una mejor gestion y un cambio politico en Madrid, estariamos dispuestos a estudiarlo, ha respondido Franco en una entrevista en la Cadena Ser recogida por Europa Press.
No obstante, el tambien delegado del Gobierno en Madrid ha afirmado que en el ambito madrileño, a diferencia del nacional, no han tenido contacto con Ciudadanos Cs, socio actual del Gobierno regional, cuyos votos son necesarios para que saliera adelante una mocion de censura. Pero si al final hay un acuerdo presupuestario en España entre Cs y PSOE e Ines Arrimadas, en ejercicio de responsabilidad y sensatez, le recomienda a Ignacio Aguado que apoye una mocion de censura, estamos preparados. Tenemos equipo, programa y tenemos las personas adecuadas para gobernar de otra forma y demostrar, de una vez por toda, que otra solucion es posible, ha esgrimido
El lider de los socialistas madrileños ha afirmado que les sobran los motivos para presentar una mocion de censura, pero cree, y la mayoria de los socialistas, que ahora no es el momento por responsabilidad ante la crisis del coronavirus. Este gobierno se merece una mocion de censura pero a dia de hoy no seria responsable presentarla con lo que esta cayendo, con la situacion de la pandemia. Todos debemos sumar fuerzas y funcionar lealmente para frenarla. Ahora mismo es el momento de la unidad para luchar contra la pandemia, ha dicho.
'''

In [40]:
lst = [text, text1]

In [41]:
df = pd.DataFrame(lst,columns=["text"])

In [42]:
df.head()

,text
0,\nPablo Casado se pronuncio finalmente sobre l...
1,"\nEl secretario general del PSOE de Madrid, Jo..."


In [43]:
### Loading the Model
#model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)

In [44]:
def get_summary(text_data,max_seq_len = 100):
    ''' Takes in n the text data and outputs the summaries.
    
        max_seq_len = the maximum sequence length that we wish our summary to have. Can be used to control the size of summary default = 100'''
    
    summaries = [] # empty list to hold all of our summaries
    for text in text_data:
        
        preprocess_text = text.strip().replace("\n","") # removing any extra spaces or line breaks present
        t5_prepared_Text = "summarize: "+preprocess_text # prepending 'summarize' to the text as guided in the T5 research paper
        tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt",max_length=512).to(device) # tokenizing
                # max_length here is 512 as that is the maximum lenght t5 can handle so if out text tokens have greater max len than 512
        # it is truncated
        
    
        summary_ids = model.generate(tokenized_text,
                                    num_beams=4, # number of beams in beam search
                                    max_length = max_seq_len, # max length of the sequence
                                    early_stopping=True) # stops beam search when number sentences equal to num_beams have finnished
        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True) # decoding the tokens to text
        # the size of summary_ids size [1,seq_len] so we do summary_ids[0] to get shape of [seq_len] 
        
        summaries.append(output) # append to list created 
    return summaries

In [45]:
summaries = get_summary(df.text,max_seq_len=100)
df['Summary'] = summaries

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [46]:
df.head()

,text,Summary
0,\nPablo Casado se pronuncio finalmente sobre l...,"al supuesto espionaje del PP a Luis Barcenas, ..."
1,"\nEl secretario general del PSOE de Madrid, Jo...","el secretario general del PSOE de Madrid, Jose..."


In [54]:
for i,column in enumerate(df['text']):
    print('Original: ',df['text'][i],'\n')
    print('Summary: ',df['Summary'][i],'\n')

Original:  
Pablo Casado se pronuncio finalmente sobre la denominada pieza Kitchen, referida al supuesto espionaje del PP a Luis Barcenas y sobre la que el juez ha levantado secreto de sumario, asegurando que son investigaciones judiciales que vienen de tiempos pasados y que reafirman su compromiso con la honorabilidad y la ejemplaridad que deben tener todos los partidos politicos”. El ahora lider de la formacion recordo que en 2012 y 2013 (años a los que se circunscriben las indagaciones) el era un diputado por avila que no pertenecia a la direccion.
Ademas, el lider del PP insistio en un foro organizado por el diario El Mundo que las personas sobre las que la Fiscalia pide una investigacion, entre los que estan el exministro del Interior, Jorge Fernandez Diaz, y la ex secretaria general del PP, Maria Dolores de Cospedal, estan fuera de la politica y no ostentan ninguna responsabilidad en el partido. Ni siquiera fueron en las listas electorales. Cospedal, de hecho, compitio con Casado